**bam indexing** (for downstream samtools view) </br>
*uncomment to run (including %%bash)

In [5]:
# %%bash
# ## symbolic link file to the bam
# ln -s /media/storage/hadas/projects/SomaticSerum/raw/pre-LUAD-02.bam -T ./data/bam_source/pre-LUAD-02.bam
# samtools index ./data/bam_source/pre-LUAD-02.bam

ln: failed to create symbolic link './data/bam_source/pre-LUAD-02.bam': File exists


**create sample bam file** (for testing)<br>
uncomment to run (including %%bash)

In [31]:
# %%bash

# # Input and otuput BAM file 
# INPUT_BAM="/home/ido/bqsr_tab_anslysis/data/bam_source/pre-LUAD-02.bam"
# INPUT_BASENAME=$(basename "$INPUT_BAM" .bam)

# # Region, Start and end positions of small genomic region
# G_REGION="1"   # => chromosome 1
# START_POS=1
# END_POS=1000000

# OUTPUT_DIR="./data/bam_source" 
# OUTPUT_BAM="${OUTPUT_DIR}/${INPUT_BASENAME}_chr${G_REGION}_${START_POS}_${END_POS}.bam"
# samtools view -b $INPUT_BAM $G_REGION:$START_POS-$END_POS > $OUTPUT_BAM
# samtools view -H $OUTPUT_BAM > "${OUTPUT_BAM}.header"
# ls -lt "${OUTPUT_DIR}/${INPUT_BASENAME}"*

-rw-rw-r-- 1 ido ido     2984 Mar 30 16:36 ./data/bam_source/pre-LUAD-02_chr1_1_1000000.bam.header
-rw-rw-r-- 1 ido ido 24968500 Mar 30 16:36 ./data/bam_source/pre-LUAD-02_chr1_1_1000000.bam
-rw-rw-r-- 1 ido ido  8897976 Mar 30 16:18 ./data/bam_source/pre-LUAD-02.bam.bai
-rw-rw-r-- 1 ido ido     2827 Mar 30 13:46 ./data/bam_source/pre-LUAD-02.bam.header
lrwxrwxrwx 1 ido ido       62 Mar 30 13:26 ./data/bam_source/pre-LUAD-02.bam -> /media/storage/hadas/projects/SomaticSerum/raw/pre-LUAD-02.bam


using **picard** to generate genome dictionary

In [9]:
# %%bash 
# REF_GENOME_FULL_PATH="${HOME}/resources/hs37d5/hs37d5.fa"
# REF_GENOME_BASENAME=$(basename "$REF_GENOME_FULL_PATH")
# REF_GENOME_NO_EXT=${REF_GENOME_BASENAME%.*}
# DICT_FILE=$(dirname "$REF_GENOME_FULL_PATH")/${REF_GENOME_NO_EXT}.dict
# PICARD_DIR=$HOME/picard/build/libs
# samtools faidx $REF_GENOME_FULL_PATH
# java -jar $PICARD_DIR/picard.jar CreateSequenceDictionary R=$REF_GENOME_FULL_PATH O=$DICT_FILE

INFO	2023-04-09 19:53:08	CreateSequenceDictionary	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** 
https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    CreateSequenceDictionary -R /home/ido/resources/hs37d5/hs37d5.fa -O /home/ido/resources/hs37d5/hs37d5.dict
**********


19:53:08.679 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/ido/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Sun Apr 09 19:53:08 IDT 2023] CreateSequenceDictionary OUTPUT=/home/ido/resources/hs37d5/hs37d5.dict REFERENCE=/home/ido/resources/hs37d5/hs37d5.fa    TRUNCATE_NAMES_AT_WHITESPACE=true NUM_SEQUENCES=2147483647 VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=fals

**indexing vcf files**</br>
*uncomment to run (including %%bash)

In [14]:
%%bash
VCF_ALL_FULL_PATH=$HOME/mapping_resources/00-All.vcf.gz
VCF_COMMON_FULL_PATH=$HOME/mapping_resources/00-common_all.vcf.gz
#samtools index $VCF_ALL_FULL_PATH
gatk IndexFeatureFile --input $VCF_ALL_FULL_PATH --verbosity ERROR &
gatk IndexFeatureFile --input $VCF_COMMON_FULL_PATH --verbosity ERROR &

Using GATK jar /home/ido/gatk/build/libs/gatk-package-4.4.0.0-9-g56693ff-SNAPSHOT-local.jarUsing GATK jar /home/ido/gatk/build/libs/gatk-package-4.4.0.0-9-g56693ff-SNAPSHOT-local.jar
Running:

Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/ido/gatk/build/libs/gatk-package-4.4.0.0-9-g56693ff-SNAPSHOT-local.jar IndexFeatureFile --input /home/ido/mapping_resources/00-All.vcf.gz --verbosity ERROR
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/ido/gatk/build/libs/gatk-package-4.4.0.0-9-g56693ff-SNAPSHOT-local.jar IndexFeatureFile --input /home/ido/mapping_resources/00-common_all.vcf.gz --verbosity ERROR
[9 April 2023 at 20:18:40 IDT] org.broadinstitute.hellbender.tools.IndexFeatureFile done. Elapsed time: 1.12 minutes.
Runtime.tot

Tool returned:
/home/ido/mapping_resources/00-common_all.vcf.gz.tbi


[9 April 2023 at 20:31:14 IDT] org.broadinstitute.hellbender.tools.IndexFeatureFile done. Elapsed time: 13.70 minutes.
Runtime.totalMemory()=654311424


Tool returned:
/home/ido/mapping_resources/00-All.vcf.gz.tbi


**BaseRecalibrator**<br>

In [15]:
%%bash
IN_BAM_FULL_PATH=$HOME/bqsr_tab_anslysis/data/bam_source/pre-LUAD-02_chr1_1_1000000.bam
IN_BAM_BASENAME=$(basename "$IN_BAM_FULL_PATH")
REF_GENOME=$HOME/resources/hs37d5/hs37d5.fa
#KNOWN_SITES=$HOME/mapping_resources/00-All.vcf.gz
KNOWN_SITES=$HOME/mapping_resources/00-common_all.vcf.gz
OUTDIR=$HOME/bqsr_tab_anslysis/data/test_bqsr
OUTFILE=$OUTDIR/$IN_BAM_BASENAME.recal_data.table
#OTHER_ARGS="--java-options '-DGATK_STACKTRACE_ON_USER_EXCEPTION=true"
BASE_RECALIRATOR_CMD="$HOME/gatk/gatk BaseRecalibrator -I $IN_BAM_FULL_PATH -R $REF_GENOME \
                    --known-sites $KNOWN_SITES -O $OUTFILE"
echo $BASE_RECALIRATOR_CMD
#$BASE_RECALIRATOR_CMD

/home/ido/gatk/gatk BaseRecalibrator -I /home/ido/bqsr_tab_anslysis/data/bam_source/pre-LUAD-02_chr1_1_1000000.bam -R /home/ido/resources/hs37d5/hs37d5.fa --known-sites /home/ido/mapping_resources/00-common_all.vcf.gz -O /home/ido/bqsr_tab_analysis/data/test_bqsr/pre-LUAD-02_chr1_1_1000000.bam.recal_data.table


### BaseRecalibrator Call
(inspired by URL  https://gist.github.com/mbk0asis/59eed2009f8185a744eb705f9e444531)
### another ref with the dbsnp: https://github.com/igsr/1000Genomes_data_indexes/blob/master/historical_data/former_toplevel/README.alignment_data.md